# Data Retrieval Direct from Source

## Installation
Open a conda terminal and install db.py

pip install db.py

In the docs for db.py, a Python 2 only module is listed as the one to use for MySQL. Instead, use the following package:

pip install mysqlclient

There are a few other options but this is the only one I've seen working

In [1]:
from db import DB
import pandas as pd
import numpy as np
import matplotlib as plt

## Establishing connection with the database
Store your db credentials in a local profile using our shared name, mariadb_ro. This way, whenever we share notebooks, our machines will all automatically use our own credentials.

In [2]:
db = DB(username="imama_ro", password="NYCnL5jJ4xSqcWU/6JUJog==", hostname="db-dw.lvh.systems",dbtype="mysql")
db.save_credentials(profile="mariadb_ro")

You only need to run this once. After that, execute the following code to load your profile

In [3]:
db = DB(profile="mariadb_ro")

# Finding Functions

In [4]:
#db.find_table("A*")

# Querying the Database

All results are automatically placed into a DataFrame.
Either run the query directly from a file or from a string 

In [5]:
#from file
#df_from_file = db.query_from_file("myscript.sql")

# Downloading the Search console data first

Note: Everything is easier to read when the query string is stored seperately and errors will be reduced when this is encapsulated in triple quotes. 

In [18]:
q1 = """
select google_search_performance.page as 'page', google_search_performance.query as 'query',
       sum(clicks) as 'clicks',
       sum(impressions) as 'impressions',
       max(impressions) as 'max_impressions',
        sum(clicks)/sum(impressions) as 'ctr',
        avg(google_search_performance.position) as 'position',
       sum(impressions* position)/sum(impressions) as 'weighted_position'
from dw.google_search_performance
where _date>=date_sub(curdate(),interval 7 day ) and query NOT LIKE '%love%'
group by 1,2
order by impressions desc ;

"""

In [19]:
df = db.query(q1)

In [20]:
df

,page,query,clicks,impressions,max_impressions,ctr,position,weighted_position
0,https://www.loveholidays.com/holidays/beach-ho...,on the beach,183.0,75270.0,11821.0,0.00243,7.897710,5.351616
1,https://www.loveholidays.com/holidays/first-ch...,first choice,24.0,27998.0,5409.0,0.00086,10.175500,9.319355
2,https://www.loveholidays.com/holidays/,holidays,421.0,23057.0,3279.0,0.01826,8.546198,7.030678
3,https://www.loveholidays.com/holidays/cheap-ho...,cheap holidays,595.0,22081.0,3700.0,0.02695,7.819351,5.463274
4,https://www.loveholidays.com/holidays/thomson-...,thomson,43.0,21956.0,2941.0,0.00196,10.302143,7.748169
...,...,...,...,...,...,...,...,...
63230,https://www.loveholidays.com/holidays/bulgaria...,sun city sunny beach,1.0,1.0,1.0,1.00000,3.000000,3.000000
63231,https://www.loveholidays.com/holidays/city-bre...,mini breaks march 2020,1.0,1.0,1.0,1.00000,14.000000,14.000000
63232,https://www.loveholidays.com/destinations/,cheap exotic holidays 2020,1.0,1.0,1.0,1.00000,11.000000,11.000000
63233,https://www.loveholidays.com/holidays/clubbing...,drinking holiday destinations,1.0,1.0,1.0,1.00000,4.000000,4.000000


In [21]:
df_page=df['page'].tolist()

In [22]:
df_page

['https://www.loveholidays.com/holidays/beach-holidays.html',
 'https://www.loveholidays.com/holidays/first-choice-holidays.html',
 'https://www.loveholidays.com/holidays/',
 'https://www.loveholidays.com/holidays/cheap-holidays.html',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/beach-holidays.html',
 'https://www.loveholidays.com/holidays/last-minute-holidays.html',
 'https://www.loveholidays.com/holidays/all-inclusive-holidays.html',
 'https://www.loveholidays.com/holidays/black-friday-holiday-deals-holidays.html',
 'https://www.loveholidays.com/holidays/fuerteventura-holidays.html',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/canary-islands-holidays.html',
 'https://www.loveholidays.com/holidays/',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/f

# Removing duplications from the list of urls 


In [23]:
def my_function(x):
  return list(dict.fromkeys(x))

page_list = my_function(df_page)
print(page_list)

['https://www.loveholidays.com/holidays/beach-holidays.html', 'https://www.loveholidays.com/holidays/first-choice-holidays.html', 'https://www.loveholidays.com/holidays/', 'https://www.loveholidays.com/holidays/cheap-holidays.html', 'https://www.loveholidays.com/holidays/thomson-holidays.html', 'https://www.loveholidays.com/holidays/last-minute-holidays.html', 'https://www.loveholidays.com/holidays/all-inclusive-holidays.html', 'https://www.loveholidays.com/holidays/black-friday-holiday-deals-holidays.html', 'https://www.loveholidays.com/holidays/fuerteventura-holidays.html', 'https://www.loveholidays.com/holidays/canary-islands-holidays.html', 'https://www.loveholidays.com/holidays/dubai-holidays.html', 'https://www.loveholidays.com/holidays/cape-verde-islands-holidays.html', 'https://www.loveholidays.com/holidays/tenerife-holidays.html', 'https://www.loveholidays.com/holidays/city-breaks-holidays.html', 'https://www.loveholidays.com/holidays/new-york-holidays.html', 'https://www.love

In [24]:
type(page_list)

list

In [25]:
page_list

['https://www.loveholidays.com/holidays/beach-holidays.html',
 'https://www.loveholidays.com/holidays/first-choice-holidays.html',
 'https://www.loveholidays.com/holidays/',
 'https://www.loveholidays.com/holidays/cheap-holidays.html',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/last-minute-holidays.html',
 'https://www.loveholidays.com/holidays/all-inclusive-holidays.html',
 'https://www.loveholidays.com/holidays/black-friday-holiday-deals-holidays.html',
 'https://www.loveholidays.com/holidays/fuerteventura-holidays.html',
 'https://www.loveholidays.com/holidays/canary-islands-holidays.html',
 'https://www.loveholidays.com/holidays/dubai-holidays.html',
 'https://www.loveholidays.com/holidays/cape-verde-islands-holidays.html',
 'https://www.loveholidays.com/holidays/tenerife-holidays.html',
 'https://www.loveholidays.com/holidays/city-breaks-holidays.html',
 'https://www.loveholidays.com/holidays/new-york-holidays.html',
 'h

# Installing headless chrome 

In [12]:
# Using headless chrome we'll extract the page title of all URLs
# Installing the relevant drivers



In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Installing the chrome driver

In [14]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(r'C:\Users\Asad\seowork\chromedriver.exe',options=chrome_options)

In [26]:
pages = page_list[0:10]

In [27]:
pages

['https://www.loveholidays.com/holidays/beach-holidays.html',
 'https://www.loveholidays.com/holidays/first-choice-holidays.html',
 'https://www.loveholidays.com/holidays/',
 'https://www.loveholidays.com/holidays/cheap-holidays.html',
 'https://www.loveholidays.com/holidays/thomson-holidays.html',
 'https://www.loveholidays.com/holidays/last-minute-holidays.html',
 'https://www.loveholidays.com/holidays/all-inclusive-holidays.html',
 'https://www.loveholidays.com/holidays/black-friday-holiday-deals-holidays.html',
 'https://www.loveholidays.com/holidays/fuerteventura-holidays.html',
 'https://www.loveholidays.com/holidays/canary-islands-holidays.html']

In [40]:
page_title= []
desc = []
for u in pages:
    driver.get(u)
#     Getting the page title
    pt = driver.title
    page_title.append(pt)
    
#     Getting the page description
    pd = driver.find_elements_by_xpath('/html/head/meta[9]')[0]
    desc.append(pd.get_attribute("content"))
    
    time.sleep(1)


In [41]:
page_title

['Beach Holidays 2019 / 2020 | Holidays from £81pp | loveholidays.com',
 'First Choice Holidays 2019 / 2020 | loveholidays.com',
 'Holidays 2019 | Holiday Search | loveholidays.com',
 'Cheap Holidays 2019 / 2020 | Holidays from £68pp | loveholidays.com',
 'Thomson Holidays 2019 / 2020 | loveholidays.com',
 'Last Minute Holidays & Late Holiday Deals 2019 / 2020 | Holidays from £68pp | loveholidays.com',
 'All Inclusive Holidays 2019 / 2020 | Holidays from £158pp | loveholidays.com',
 'Black Friday Holiday Deals 2019 / 2020 | loveholidays.com',
 'Fuerteventura Holidays 2019 / 2020 | Holidays from £153pp | loveholidays.com',
 'Canary Islands Holidays 2019 / 2020 | Holidays from £126pp | loveholidays.com']

In [42]:
desc

["Picking the right beach holiday is an incredibly serious business, we'll help. Book now with low deposits, easy pay monthly options and ABTA & ATOL protection.",
 'Looking for the best deals on First Choice Holidays? Low Deposit & Monthly Payments. Book securely with loveholidays™.',
 'Loveholidays',
 'Finding your cheap holiday is easy. Save money with great deals, low deposits & monthly payments options available. Book securely with loveholidays™.',
 'Looking for the best deals on Thomson Holidays? Low Deposit & Monthly Payments. Book securely with loveholidays™.',
 'Discover a great choice of last minute holidays at amazing low prices. Book now with low deposits, easy pay monthly options and ABTA & ATOL protection.',
 'All inclusive holidays allow you to indulge without any unexpected expense. Book now with low deposits, easy pay monthly options and ABTA & ATOL protection.',
 'Check out our best deals for Black Friday & Cyber Monday!',
 'Fuerteventura offers absolute buckets of su

In [46]:
meta_df = pd.DataFrame({'title':page_title,'desc':desc})

AttributeError: 'WebElement' object has no attribute 'DataFrame'